In [ ]:
import pandas as pd
import numpy as np
#from tbdynamics.plotting import plot_model_vs_actual
import nevergrad as ng

# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model
from tb_incubator.plotting import set_plot_label, display_plot, plot_model_vs_actual
from tb_incubator.input import load_targets, load_param_info, get_param_table
from tb_incubator.model import build_model
from multiprocessing import cpu_count
pd.options.plotting.backend = "plotly"

from estival import targets as est
from estival import priors as esp
from estival.model import BayesianCompartmentalModel
from estival.wrappers import pymc as epm
import pymc as pm
import tb_incubator.constants as const

In [2]:
param_info = load_param_info()
params = param_info["value"]

compartments = const.compartments
infectious_compartments = const.infectious_compartments
age_strata = const.age_strata
model_times = const.model_times

model, desc = build_model(
    compartments,
    infectious_compartments,
    age_strata,
    params,
    model_times
)

model.run(params)

In [3]:
all_targets = load_targets()

In [4]:
targets = [
    est.TruncatedNormalTarget("prevalence", all_targets["prevalence"], (0.0, np.inf),
        esp.UniformPrior("prevalence_dispersion", (0.1, all_targets["prevalence"].max() * 0.1))
    ),
    est.TruncatedNormalTarget("notification", all_targets["notif2000"], (0.0, np.inf),
        esp.UniformPrior("notification_dispersion", (0.1, all_targets["notif2000"].max() * 0.1))
    )
]

priors = [
    esp.UniformPrior("contact_rate", (0.01, 3.0)),
    esp.UniformPrior("self_recovery_rate", (0.05, 0.30)),
    esp.UniformPrior("screening_scaleup_shape", (0.01, 0.3)),
    esp.UniformPrior("screening_inflection_time", (1990.0, 2020.0)),
    esp.UniformPrior("time_to_screening_end_asymp", (0.1, 20.0)),
    esp.UniformPrior("rr_infection_latent", (0.01, 1.0)),
    esp.UniformPrior("rr_infection_recovered", (0.01, 1.0)),
    esp.UniformPrior("seed_time", (1840.0, 1900.0)),
    esp.UniformPrior("seed_duration", (1.0, 20.0)),
    esp.UniformPrior("seed_rate", (1.0, 100.0)),
]

bcm = BayesianCompartmentalModel(model, params, priors, targets)

In [5]:
def calibrate_model_with_optimisation(bcm):
    """
    This function performs a model calibration using optimisation. 
    Calibration is performed using the estival package, which implements a wrapper for optimisation methods provided by the nevergrad package. 

    Args:
        bcm: the calibration model object (type BayesianCompartmentalModel) 
    """
    # create a nevergrad optimisation runner
    from nevergrad.optimization.differentialevolution import TwoPointsDE
    orunner = optimize_model(bcm, opt_class=TwoPointsDE, num_workers=cpu_count(), budget=4000)
    # perform optimisation, allowing for up to 1000 model evaluations
    rec = orunner.minimize(4000)
    # retrieve optimised parameter values
    optimised_params = rec.value[1]    
   
    return optimised_params

In [ ]:
optimised_params = calibrate_model_with_optimisation(bcm)
optimised_params

In [7]:
# run the modle with the optimised parameter set
res = bcm.run(params | optimised_params)

In [8]:
derived_df_0 = res.derived_outputs

In [ ]:
plot = plot_model_vs_actual(
    derived_df_0, all_targets['notif'], "notification", "Notification", "Modelled vs Data"
)

plot.update_xaxes(range=[1978, 2023])

In [10]:
def calibrate_model_with_sampling(bcm):
    """
    This function performs a model calibration using Bayesian sampling. 
    Calibration is performed using the estival package, which implements a wrapper for sampling methods provided by the PyMC package. 

    Args:
        bcm: the calibration model object (type BayesianCompartmentalModel) 
    """

    with pm.Model() as model:
        # This is all you need - a single call to use_model
        variables = epm.use_model(bcm)

        # Now call a sampler using the variables from use_model
        # In this case we use the Differential Evolution Metropolis(Z) sampler
        # See the PyMC docs for more details
        idata = pm.sample(step=[pm.DEMetropolisZ(variables)], draws=2000, tune=2000,cores=4,chains=4)
    
    return idata

In [ ]:
# Set the number of workers for parallel optimization
orunner = optimize_model(bcm, opt_class=ng.optimizers.TwoPointsDE, num_workers=cpu_count())
for i in range(8):
    rec = orunner.minimize(1000)
mle_params = rec.value[1]
mle_params

In [12]:
res = bcm.run(params | mle_params)
derived_df_0 = res.derived_outputs

In [ ]:
plot = plot_model_vs_actual(
    derived_df_0, all_targets['notif'], "notification", "Notification", "Modelled vs Data"
)

plot.update_xaxes(range=[1978, 2023])